In [1]:
#packages
import os
import numpy as np
import pandas as pd

In [2]:
#data source: https://www.kaggle.com/c/two-sigma-financial-news/data
#read in data
market = pd.read_csv('market.csv')
news = pd.read_csv('news.csv')

In [3]:
#format time column as yyyy-mm-dd
market['time'] = market['time'].str.split(' ', expand = True)[0]
news['time'] = news['time'].str.split(' ', expand = True)[0]
#select 2007-02-05, 2007-03-05, 2007-04-05, 2007-05-05, 2007-06-05, and 2007-07-05
market = market[market['time'].isin(['2007-02-05', '2007-03-05', '2007-04-05', '2007-05-05', '2007-06-05', '2007-07-05', 
                                    '2007-02-25', '2007-03-25', '2007-04-25', '2007-05-25', '2007-06-25', '2007-07-25'])]
news = news[news['time'].isin(['2007-02-05', '2007-03-05', '2007-04-05', '2007-05-05', '2007-06-05', '2007-07-05',
                              '2007-02-25', '2007-03-25', '2007-04-25', '2007-05-25', '2007-06-25', '2007-07-25'])]

In [4]:
#drop unnecessary columns
market = market.drop('Unnamed: 0', axis = 1)
news = news.drop('Unnamed: 0', axis = 1)
#select columns of interest
market = market.loc[:, ['time', 'assetCode', 'assetName', 'volume', 'open', 'close']]
news = news.loc[:, ['time', 'assetCodes', 'assetName', 'provider', 'relevance', 'urgency', 'sentimentClass', 'sentimentNegative', 'sentimentNeutral', 'sentimentPositive', 'firstMentionSentence', 'bodySize', 'wordCount', 'sentenceCount']]
#create result and binary result columns in market
market['result'] = market['close'] - market['open']
market['binary_result'] = 0
market.loc[market['result'] >= 0, ['binary_result']] = 1
market.loc[market['result'] < 0, ['binary_result']] = 0
#create asset code column in news
ac = news.assetCodes.str.split(",").apply(pd.Series, 1).stack()
ac.index = ac.index.droplevel(-1)
ac = ac.str.strip('{}')
ac = ac.str.strip("'")
ac = ac.str.strip("' ")
ac.name = 'assetCode'
news = news.drop('assetCodes', axis = 1)
news = news.join(ac)
cols = news.columns.tolist()
cols = ['time','assetCode','assetName','provider','sentimentClass','sentimentNegative', 'sentimentNeutral','sentimentPositive','urgency','firstMentionSentence','bodySize','wordCount','sentenceCount']
news = news[cols]

In [5]:
#combine market and news
df = pd.merge(market, news, how = 'outer', on = ['time', 'assetCode']).dropna()
df = df.drop('assetName_y', axis = 1)
df = df.rename(columns = {'assetName_x': 'assetName'})
df.head()

,time,assetCode,assetName,volume,open,close,result,binary_result,provider,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,urgency,firstMentionSentence,bodySize,wordCount,sentenceCount
0,2007-02-05,A.N,Agilent Technologies Inc,2512100.0,31.64,31.83,0.19,1.0,BSW,1.0,0.079699,0.271277,0.649025,3.0,1.0,5393.0,843.0,34.0
3,2007-02-05,AAPL.O,Apple Inc,20737742.0,84.30,83.94,-0.36,0.0,PRN,1.0,0.123388,0.161730,0.714882,3.0,1.0,1757.0,352.0,17.0
4,2007-02-05,AAPL.O,Apple Inc,20737742.0,84.30,83.94,-0.36,0.0,RTRS,1.0,0.029522,0.117037,0.853441,1.0,1.0,0.0,13.0,1.0
5,2007-02-05,AAPL.O,Apple Inc,20737742.0,84.30,83.94,-0.36,0.0,RTRS,-1.0,0.419389,0.163392,0.417219,1.0,1.0,0.0,37.0,1.0
6,2007-02-05,AAPL.O,Apple Inc,20737742.0,84.30,83.94,-0.36,0.0,RTRS,-1.0,0.386227,0.355714,0.258059,1.0,1.0,0.0,24.0,1.0


In [6]:
#write data to csv
df.to_csv('data.csv')